In [1]:
%load_ext autoreload
%autoreload 2

import datetime
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm
from tqdm.contrib import tzip, tenumerate, tmap

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
import pyspark.sql.types as pstype
import pyspark.sql.functions as F
import pyspark as ps

import matplotlib as mlt
import matplotlib.pyplot as plt
import japanize_matplotlib

from time_series_model import *

%matplotlib inline
%matplotlib ipympl

In [22]:
np.set_printoptions(threshold=100000, precision=4, linewidth=10000)
ps_conf = ps.SparkConf().set("spark.logConf", "false")\
            .set("spark.executor.memory", "12g")\
            .set("spark.driver.memory", "4g")\
            .set("spark.executor.cores", "7")\
            .set("spark.sql.shuffle.partitions", "500")\
            .set("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+UseStringDeduplication")\
            .set("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "G1 Young Generation")\
            .set("spark.eventLog.gcMetrics.oldGenerationGarbageCollectors", "G1 Old Generation")\
			.set("spark.logConf", "false")
spark = SparkSession.builder.config(conf=ps_conf).getOrCreate()

In [23]:
SPECIFIED_PATH = "csv_data/"
SPECIFIED_DATE = "20241120"
SPECIFIED_CSV  = SPECIFIED_PATH + SPECIFIED_DATE

In [ ]:
input_path  = SPECIFIED_CSV + "_raw_data_sumida_bunka.csv"
df_raw_data = spark.read.option("inferSchema", "True").option("header", "True").csv(input_path)
df_raw_data.persist(StorageLevel.MEMORY_AND_DISK_DESER)

utid_list = sorted(df_raw_data.select("unit_id").drop_duplicates().rdd.flatMap(lambda x: x).collect())

In [ ]:
df_raw_data\
		.filter(col("unit_id").isin(utid_list))\
		.filter(col("randomized") == 1)\
		.select(['date', 'datetime', 'unit_id', 'aibeaconid'])\
        .withColumn('minute', F.date_trunc('minute', 'datetime'))\
        .withColumn('minute', F.unix_timestamp('minute'))\
        .withColumn('minute', col('minute') - (col('minute') % 60))\
        .withColumn('minute', F.from_unixtime('minute'))\
        .select(['date', 'minute', 'unit_id', 'aibeaconid'])\
        .drop_duplicates()\
        .groupBy(['date', 'minute', 'unit_id'])\
        .agg(F.count('*').alias('1min_count'))\
        .orderBy(['date', 'minute', 'unit_id'])\
        .toPandas()\
        .to_csv(SPECIFIED_CSV + "_1min_data_sumida_bunka.csv", header=True, index=False)

In [ ]:
SPECIFIED_PAST = (datetime.datetime.strptime(SPECIFIED_DATE, '%Y%m%d') - relativedelta(days=1) + datetime.timedelta(hours=9)).strftime('%Y-%m-%d')

start_time = f'{SPECIFIED_PAST} 00:00:00'
end_time = f'{SPECIFIED_PAST} 23:59:00'
# 1分単位の時間列を作成
time_range = pd.date_range(start=start_time, end=end_time, freq='min')
# DataFrameに変換
df_time = pd.DataFrame(time_range, columns=['datetime'])
df_time = spark.createDataFrame(df_time)\
				.withColumn('hour', F.hour(col('datetime')))

df_by1min = spark.read\
				.option('header', True)\
				.option('inferSchema', True)\
           		.csv(SPECIFIED_CSV + "_1min_data_sumida_bunka.csv")

for unit_id in utid_list:
    df_tmp  = df_by1min\
        		.filter(col('unit_id') == unit_id)\
          		.select(['minute', '1min_count'])\
            	.withColumnRenamed('minute',     'datetime')\
                .withColumnRenamed('1min_count', unit_id)
    df_time = df_time.join(df_tmp, on='datetime', how='left')
df_time = df_time\
			.fillna(0)\
    		.orderBy('datetime')
df_time.persist(StorageLevel.MEMORY_AND_DISK_DESER)
df_time.show()

In [ ]:
SPECIFIED_HOUR = 14

pd_data = df_time.toPandas()
pd_data = pd_data[pd_data['hour'] == SPECIFIED_HOUR]
pd_data = pd_data[utid_list]
pd_data

In [ ]:
test_data = pd_data.values.tolist()
test_data

In [ ]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="external library", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし
# 総実行時間:  6.6s
# 平均二乗誤差(MSE): 8.716582787838323
# L2正則化項(l2 norm): 38368.89051812161
# L1正則化項(l1 norm): 428.0689097535977
# 目的関数(Objective):  1938830.5182206288
# 目的関数(Objective)の微分:  833114002.0581789
# 実データとの平均二乗誤差:  8.716582787838323

# 標準化処理
# 総実行時間:  0.0s
# 平均二乗誤差(MSE): 43.17262540588774
# L2正則化項(l2 norm): 1.2181558529806789
# L1正則化項(l1 norm): 35.530871686969626
# 目的関数(Objective):  1668.1743292491428
# 目的関数(Objective)の微分:  81626.36404631077
# 実データとの平均二乗誤差:  6983.460640955819

In [ ]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="coordinate descent", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean = model.predict(x_tmp)

# print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし
# 総実行時間:  3m 6.5s
# 平均二乗誤差(MSE): 8.71737841354076
# L2正則化項(l2 norm): 38401.28543962791
# L1正則化項(l1 norm): 428.062500467189
# 目的関数(Objective):  1940463.172758058
# 目的関数(Objective)の微分:  833428059.4004015
# 実データとの平均二乗誤差:  8.71737841354076

# 標準化処理
# 総実行時間:  32.7s
# 平均二乗誤差(MSE): 0.7828437567850707
# L2正則化項(l2 norm): 101.66168589323144
# L1正則化項(l1 norm): 578.1477689441324
# 目的関数(Objective):  11620.92360638313
# 目的関数(Objective)の微分:  148214.0310893795
# 実データとの平均二乗誤差:  8.716582787396604

In [ ]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="ISTA", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし(収束せず)
# 総実行時間:  29.0s
# 平均二乗誤差(MSE): 5.750641650258252
# L2正則化項(l2 norm): 199.86358106593997
# L1正則化項(l1 norm): 1258.9142527745976
# 目的関数(Objective):  24333.9820830061
# 目的関数(Objective)の微分:  19668694.11350823
# 実データとの平均二乗誤差:  5.750641650258252

# 標準化処理
# 総実行時間:  29.7s
# 平均二乗誤差(MSE): 0.7828438092156231
# L2正則化項(l2 norm): 101.66107323195446
# L1正則化項(l1 norm): 578.1481380987171
# 目的関数(Objective):  11620.896864254175
# 目的関数(Objective)の微分:  148214.0684396389
# 実データとの平均二乗誤差:  8.716582912358936

In [ ]:
model = Sparse_Vector_Auto_Regressive(test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="FISTA", visible_flg=True)

x_tmp = np.array([np.array(test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(test_data))])
y_tmp = test_data[lag:]
mean  = model.predict(x_tmp)

# print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし(収束せず)
# 総実行時間:  29.9s
# 平均二乗誤差(MSE): 8.704095671225188
# L2正則化項(l2 norm): 45931.67959324515
# L1正則化項(l1 norm): 428.2588855306884
# 目的関数(Objective):  2319996.8656962933
# 目的関数(Objective)の微分:  908352255.7447977
# 実データとの平均二乗誤差:  8.704095671225186

# 標準化処理
# 総実行時間:  1.1s
# 平均二乗誤差(MSE): 0.7828414135642455
# L2正則化項(l2 norm): 101.66393873223849
# L1正則化項(l1 norm): 578.1515745664261
# 目的関数(Objective):  11621.079706828592
# 目的関数(Objective)の微分:  148214.59979816235
# 実データとの平均二乗誤差:  8.716707219281101

In [33]:
SPECIFIED_PATH = "csv_data/"
SPECIFIED_DATE = "2024-11-13"
SPECIFIED_CSV  = SPECIFIED_PATH + SPECIFIED_DATE

pd_test_data = spark.read\
				.option('header', True)\
				.option('inferSchema', True)\
           		.csv(SPECIFIED_CSV + "_time_series_data.csv")\
				.toPandas()

In [34]:
mean_sorted_list = pd_test_data.mean().sort_values(ascending=False).index
pd_test_data     = pd_test_data[mean_sorted_list].fillna(0).reset_index(drop=True)
pd_test_data     = pd_test_data.values.tolist()

In [ ]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="external library", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし
# 総実行時間:  2m 20.6s
# 平均二乗誤差(MSE): 134.83706015819152
# L2正則化項(l2 norm): 99.04386846809814
# L1正則化項(l1 norm): 744.1794175681346
# 目的関数(Objective):  86731.86624077914
# 目的関数(Objective)の微分:  2299273.858326765
# 実データとの平均二乗誤差:  134.83706015819152

# 標準化処理
# 総実行時間:  28.4s
# 平均二乗誤差(MSE): 21.281937812440326
# L2正則化項(l2 norm): 1.05124916124762
# L1正則化項(l1 norm): 48.28878953726302
# 目的関数(Objective):  6033.537700697958
# 目的関数(Objective)の微分:  598894.5664998213
# 実データとの平均二乗誤差:  10001.5034140669

In [ ]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="coordinate descent", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし
# 総実行時間:  27.3s
# 平均二乗誤差(MSE): 134.89665731706688
# L2正則化項(l2 norm): 99.10091636674454
# L1正則化項(l1 norm): 744.2241889880453
# 目的関数(Objective):  86757.45349688438
# 目的関数(Objective)の微分:  2307939.8013202604
# 実データとの平均二乗誤差:  134.89665731706683

# 標準化処理
# 総実行時間:  2m 21.8s
# 平均二乗誤差(MSE): 1.071962718158472
# L2正則化項(l2 norm): 213.9596636929814
# L1正則化項(l1 norm): 951.2089432836356
# 目的関数(Objective):  108868.97672441506
# 目的関数(Objective)の微分:  1149519.5982524685
# 実データとの平均二乗誤差:  134.83706012732063

In [ ]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="ISTA", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし(収束せず)
# 総実行時間:  51.4s
# 平均二乗誤差(MSE): 138.88829681969798
# L2正則化項(l2 norm): 106.92122778475898
# L1正則化項(l1 norm): 769.6755101206119
# 目的関数(Objective):  90768.77294503227
# 目的関数(Objective)の微分:  23090660.73588205
# 実データとの平均二乗誤差:  138.88829681969798

# 標準化処理(収束せず)
# 総実行時間:  56.9s
# 平均二乗誤差(MSE): 1.0753778023713838
# L2正則化項(l2 norm): 196.4639054332722
# L1正則化項(l1 norm): 940.7767070151838
# 目的関数(Objective):  103804.99483514356
# 目的関数(Objective)の微分:  1156503.6952151803
# 実データとの平均二乗誤差:  136.32291092456677

In [ ]:
model = Sparse_Vector_Auto_Regressive(pd_test_data, norm_α=2, l1_ratio=0.1, tol=1e-10, max_iterate=300000, isStandardization=True)

lag = 4
model.fit(lags=lag, solver="FISTA", visible_flg=True)

x_tmp = np.array([np.array(pd_test_data)[t-lag : t][::-1].ravel() for t in range(lag, len(pd_test_data))])
y_tmp = pd_test_data[lag:]
mean  = model.predict(x_tmp)

#print(f"alpha0:{model.alpha0}\nalpha:{model.alpha}", flush=True)
mse = np.sum((y_tmp - mean) ** 2) / len(y_tmp)
print(f"実データとの平均二乗誤差:  {mse}")

# 前処理なし(収束せず)
# 総実行時間:  54.0s
# 平均二乗誤差(MSE): 134.83705168697762
# L2正則化項(l2 norm): 99.04386698521003
# L1正則化項(l1 norm): 744.1793183126821
# 目的関数(Objective):  86731.85902113085
# 目的関数(Objective)の微分:  2299099.280296025
# 実データとの平均二乗誤差:  134.83705168697762

# 標準化処理
# 総実行時間:  14.3s
# 平均二乗誤差(MSE): 1.0719620369230483
# L2正則化項(l2 norm): 213.95993789196797
# L1正則化項(l1 norm): 951.2111224205046
# 目的関数(Objective):  108869.17048791474
# 目的関数(Objective)の微分:  1149520.7158008118
# 実データとの平均二乗誤差:  134.83789705893594

In [ ]:
SPECIFIED_PATH = "csv_data/"
SPECIFIED_DATE = "20240918"
SPECIFIED_CSV  = SPECIFIED_PATH + SPECIFIED_DATE

In [ ]:
input_path = SPECIFIED_CSV + "_c-united_config.csv"
df_config  = spark.read.option("inferSchema", "True").option("header", "True").csv(input_path)
df_config.persist(StorageLevel.MEMORY_AND_DISK_DESER)

utid_list = df_config.select("unit_id").drop_duplicates().rdd.flatMap(lambda x: x).collect()
spid_list = df_config.select("shop_id").drop_duplicates().rdd.flatMap(lambda x: x).collect()

In [ ]:
#pos data 前処理
input_path  = SPECIFIED_CSV + "_pos_data_table.csv"
df_pos_data = spark.read.option('inferSchema', 'True').option('header', 'True').csv(input_path)\
				.select(
					"shop_id",
                    "レシートＮｏ．",
                    "商品種別",
                    "商品コード",
                    F.regexp_replace(col("商品名称（または券名称）"), "[ 　]", "").alias("商品名称（または券名称）"),
                    "オーダー時刻",
                    "単価",
                    "数量",
                    "合計金額",
                    "date"
				)\
				.filter(col("商品名称（または券名称）") != "")\
				.groupBy("shop_id", "date", "レシートＮｏ．").agg(
                    F.last("オーダー時刻").alias("オーダー時刻"),
                    F.sum(F.when(col("商品種別") == "Y", 1).otherwise(0)).alias("レシートあたりのセット商品の数"),
                    F.sum("数量").alias("総売上点数"),
                    F.sum("合計金額").alias("総売上"),
				)\
                .withColumn("レシートあたりのセット商品の数", F.when(col("レシートあたりのセット商品の数") == 0, 1)
                            								.otherwise(col("レシートあたりのセット商品の数")))\
                .withColumn("オーダー時刻", (F.col("オーダー時刻") / 100).cast("int"))\
                .withColumnRenamed("レシートあたりのセット商品の数", "来店者数")\
                .withColumnRenamed("オーダー時刻", "hour")
df_pos_data = df_pos_data.groupBy("shop_id", "date", "hour").agg(
                    F.sum("来店者数").alias("来店者数"),
                    F.sum("総売上点数").alias("総売上点数"),
                    F.sum("総売上").alias("総売上"),
				)\
                .select(["shop_id", "date", "hour", "来店者数", "総売上点数", "総売上"])\
                .orderBy(col("shop_id").asc(), col("date").asc(), col("hour").asc())
df_pos_data.show()

In [ ]:
df_pos_data = df_pos_data\
    				.withColumn("date", F.from_unixtime(F.unix_timestamp("date") + F.col("hour") * 3600))\
                    .drop("hour")\
                    .orderBy(col("shop_id").asc(), col("date").asc())
df_pos_data.show()

In [ ]:
df_pos_data = df_pos_data\
					.join(df_config.select(["shop_id", "caption"]), on="shop_id", how="inner")\
                    .select(["shop_id", "caption", "date", "来店者数", "総売上点数", "総売上"])\
                    .orderBy(col("shop_id").asc(), col("date").asc())
df_pos_data.show()

In [ ]:
pd_pos_data  = df_pos_data.select(["shop_id", "caption", "date", "来店者数"]).toPandas()
pd_tmp_data1 = pd_pos_data[pd_pos_data["shop_id"] == 1189] # カフェ・ド・クリエグランサンシャイン通り店
pd_tmp_data2 = pd_pos_data[pd_pos_data["shop_id"] == 1616] # カフェ・ド・クリエ日比谷通り内幸町店
pd_tmp_data3 = pd_pos_data[pd_pos_data["shop_id"] == 1428] # カフェ・ド・クリエ札幌オーロラタウン店
pd_tmp_data4 = pd_pos_data[pd_pos_data["shop_id"] == 1550] # カフェ・ド・クリエ博多大博通店
pd_pos_data  = pd.merge(pd_tmp_data1, pd_tmp_data2, on="date", how="inner", suffixes=['_1', '_2'])
pd_pos_data  = pd.merge(pd_pos_data,  pd_tmp_data3, on="date", how="inner", suffixes=['_2', '_3'])
pd_pos_data  = pd.merge(pd_pos_data,  pd_tmp_data4, on="date", how="inner", suffixes=['_3', '_4'])
pd_pos_data  = pd_pos_data[["date", "来店者数_1", "来店者数_2", "来店者数_3", "来店者数_4"]]
pd_pos_data  = pd_pos_data.rename(columns={
    								"来店者数_1": "カフェ・ド・クリエグランサンシャイン通り店",
                                    "来店者数_2": "カフェ・ド・クリエ日比谷通り内幸町店",
                                    "来店者数_3": "カフェ・ド・クリエ札幌オーロラタウン店",
                                    "来店者数_4": "カフェ・ド・クリエ博多大博通店"
                                })
pd_pos_data  = pd_pos_data[[
    				"date",
                    "カフェ・ド・クリエグランサンシャイン通り店",
                    "カフェ・ド・クリエ日比谷通り内幸町店",
                    "カフェ・ド・クリエ札幌オーロラタウン店",
                    "カフェ・ド・クリエ博多大博通店"
                ]]
del pd_tmp_data1
del pd_tmp_data2
del pd_tmp_data3
del pd_tmp_data4
pd_pos_data

In [ ]:
x_data = pd_pos_data[["カフェ・ド・クリエグランサンシャイン通り店", "カフェ・ド・クリエ日比谷通り内幸町店", "カフェ・ド・クリエ札幌オーロラタウン店", "カフェ・ド・クリエ博多大博通店"]].values.tolist()
x_train, x_test = x_data[0:3600], x_data[3600:]

In [ ]:
model  = Sparse_Vector_Auto_Regressive(x_test, norm_α=0.1, l1_ratio=0.1)

lag = 4
model.fit(lags=lag)

num = len(x_test)
mean = []
for idx in range(lag, num):
	test_data = []
	for idx2 in range(0, lag):
		test_data = test_data + x_test[idx - idx2 - 1]
	tmp = model.predict([test_data])
	mean.append(tmp[0])

x_test, mean = np.array(x_test), np.array(mean)
plt.figure(figsize=(10, 12))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(4,1,1)
plt.plot(x_test[lag:, 0], "-", color="green", label="生データ")
plt.plot(mean[:, 0],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエグランサンシャイン通り店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,2)
plt.plot(x_test[lag:, 1], "-", color="green", label="生データ")
plt.plot(mean[:, 1],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ日比谷通り内幸町店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,3)
plt.plot(x_test[lag:, 2], "-", color="green", label="生データ")
plt.plot(mean[:, 2],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ札幌オーロラタウン店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,4)
plt.plot(x_test[lag:, 3], "-", color="green", label="生データ")
plt.plot(mean[:, 3],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ博多大博通店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()


model.stat_inf()

In [ ]:
lag = model.select_order(maxlag=20, ic="hqic", isVisible=True)
lag = 4
model.fit(lags=lag)

In [ ]:
x_train, x_test = x_data[0:3600], x_data[3600:]
num = len(x_test)
mean = []
test_data = []
for idx in range(0, lag):
	test_data = test_data + x_test[lag - idx - 1]
for idx in range(lag, num):
	tmp = model.predict([test_data])
	mean.append(tmp[0])
	test_data = [*tmp[0], *test_data[0:-4]]

x_plot, mean = np.array(x_test), np.array(mean)
plt.figure(figsize=(10, 12))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(4,1,1)
plt.plot(x_plot[lag:, 0], "-", color="green", label="生データ")
plt.plot(mean[:, 0],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエグランサンシャイン通り店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,2)
plt.plot(x_plot[lag:, 1], "-", color="green", label="生データ")
plt.plot(mean[:, 1],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ日比谷通り内幸町店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,3)
plt.plot(x_plot[lag:, 2], "-", color="green", label="生データ")
plt.plot(mean[:, 2],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ札幌オーロラタウン店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

plt.subplot(4,1,4)
plt.plot(x_plot[lag:, 3], "-", color="green", label="生データ")
plt.plot(mean[:, 3],      "-", color="red",   label="予測データ")
plt.title("カフェ・ド・クリエ博多大博通店")
plt.ylabel("来店者数")
plt.grid(True)
plt.legend()

model.stat_inf()

In [ ]:
print("グレンジャー因果性テスト(p値 = 0.05)")
print("")

t = model.test_causality(causing=0, caused=1)
print("カフェ・ド・クリエグランサンシャイン通り店" + " → " + "カフェ・ド・クリエ日比谷通り内幸町店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=0, caused=2)
print("カフェ・ド・クリエグランサンシャイン通り店" + " → " + "カフェ・ド・クリエ札幌オーロラタウン店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=0, caused=3)
print("カフェ・ド・クリエグランサンシャイン通り店" + " → " + "カフェ・ド・クリエ博多大博通店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=1, caused=0)
print("カフェ・ド・クリエ日比谷通り内幸町店" + " → " + "カフェ・ド・クリエグランサンシャイン通り店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=1, caused=2)
print("カフェ・ド・クリエ日比谷通り内幸町店" + " → " + "カフェ・ド・クリエ札幌オーロラタウン店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=1, caused=3)
print("カフェ・ド・クリエ日比谷通り内幸町店" + " → " + "カフェ・ド・クリエ博多大博通店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=2, caused=0)
print("カフェ・ド・クリエ札幌オーロラタウン店" + " → " + "カフェ・ド・クリエグランサンシャイン通り店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=2, caused=1)
print("カフェ・ド・クリエ札幌オーロラタウン店" + " → " + "カフェ・ド・クリエ日比谷通り内幸町店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=2, caused=3)
print("カフェ・ド・クリエ札幌オーロラタウン店" + " → " + "カフェ・ド・クリエ博多大博通店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=3, caused=0)
print("カフェ・ド・クリエ博多大博通店" + " → " + "カフェ・ド・クリエグランサンシャイン通り店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=3, caused=1)
print("カフェ・ド・クリエ博多大博通店" + " → " + "カフェ・ド・クリエ日比谷通り内幸町店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

t = model.test_causality(causing=3, caused=2)
print("カフェ・ド・クリエ博多大博通店" + " → " + "カフェ・ド・クリエ札幌オーロラタウン店")
print("F検定量:", t[0], "  ", "p値:", t[1])
print("グレンジャー因果性有り" if t[1] < 0.05 else "グレンジャー因果性無し")
print("")

In [ ]:
irf = model.irf(period=30, orth=False)

print("簡易対応表")
print("y1: グランサンシャイン通り店")
print("y2: 日比谷通り内幸町店")
print("y3: 札幌オーロラタウン店")
print("y4: 博多大博通店")

plt.figure(figsize=(10, 12))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(4,4,1)
plt.plot(irf[:, 0, 0], "-", color="green", label="生データ")
plt.title("y1 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,2)
plt.plot(irf[:, 0, 1], "-", color="green", label="生データ")
plt.title("y2 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,3)
plt.plot(irf[:, 0, 2], "-", color="green", label="生データ")
plt.title("y3 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,4)
plt.plot(irf[:, 0, 3], "-", color="green", label="生データ")
plt.title("y4 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,5)
plt.plot(irf[:, 1, 0], "-", color="green", label="生データ")
plt.title("y1 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,6)
plt.plot(irf[:, 1, 1], "-", color="green", label="生データ")
plt.title("y2 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,7)
plt.plot(irf[:, 1, 2], "-", color="green", label="生データ")
plt.title("y3 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,8)
plt.plot(irf[:, 1, 3], "-", color="green", label="生データ")
plt.title("y4 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,9)
plt.plot(irf[:, 2, 0], "-", color="green", label="生データ")
plt.title("y1 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,10)
plt.plot(irf[:, 2, 1], "-", color="green", label="生データ")
plt.title("y2 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,11)
plt.plot(irf[:, 2, 2], "-", color="green", label="生データ")
plt.title("y3 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,12)
plt.plot(irf[:, 2, 3], "-", color="green", label="生データ")
plt.title("y4 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,13)
plt.plot(irf[:, 3, 0], "-", color="green", label="生データ")
plt.title("y1 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,14)
plt.plot(irf[:, 3, 1], "-", color="green", label="生データ")
plt.title("y2 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,15)
plt.plot(irf[:, 3, 2], "-", color="green", label="生データ")
plt.title("y3 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,16)
plt.plot(irf[:, 3, 3], "-", color="green", label="生データ")
plt.title("y4 → y4")
plt.xlabel("ラグ")
plt.grid(True)

In [ ]:
irf = model.irf(period=30, orth=True)

print("簡易対応表")
print("y1: グランサンシャイン通り店")
print("y2: 日比谷通り内幸町店")
print("y3: 札幌オーロラタウン店")
print("y4: 博多大博通店")

plt.figure(figsize=(10, 12))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(4,4,1)
plt.plot(irf[:, 0, 0], "-", color="green", label="生データ")
plt.title("y1 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,2)
plt.plot(irf[:, 0, 1], "-", color="green", label="生データ")
plt.title("y2 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,3)
plt.plot(irf[:, 0, 2], "-", color="green", label="生データ")
plt.title("y3 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,4)
plt.plot(irf[:, 0, 3], "-", color="green", label="生データ")
plt.title("y4 → y1")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,5)
plt.plot(irf[:, 1, 0], "-", color="green", label="生データ")
plt.title("y1 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,6)
plt.plot(irf[:, 1, 1], "-", color="green", label="生データ")
plt.title("y2 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,7)
plt.plot(irf[:, 1, 2], "-", color="green", label="生データ")
plt.title("y3 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,8)
plt.plot(irf[:, 1, 3], "-", color="green", label="生データ")
plt.title("y4 → y2")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,9)
plt.plot(irf[:, 2, 0], "-", color="green", label="生データ")
plt.title("y1 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,10)
plt.plot(irf[:, 2, 1], "-", color="green", label="生データ")
plt.title("y2 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,11)
plt.plot(irf[:, 2, 2], "-", color="green", label="生データ")
plt.title("y3 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,12)
plt.plot(irf[:, 2, 3], "-", color="green", label="生データ")
plt.title("y4 → y3")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,13)
plt.plot(irf[:, 3, 0], "-", color="green", label="生データ")
plt.title("y1 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,14)
plt.plot(irf[:, 3, 1], "-", color="green", label="生データ")
plt.title("y2 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,15)
plt.plot(irf[:, 3, 2], "-", color="green", label="生データ")
plt.title("y3 → y4")
plt.xlabel("ラグ")
plt.grid(True)

plt.subplot(4,4,16)
plt.plot(irf[:, 3, 3], "-", color="green", label="生データ")
plt.title("y4 → y4")
plt.xlabel("ラグ")
plt.grid(True)
plt.show()

In [ ]:
periods = 30
fevd = model.fevd(period=periods)
periods = len(fevd)

print("簡易対応表")
print("y1: グランサンシャイン通り店")
print("y2: 日比谷通り内幸町店")
print("y3: 札幌オーロラタウン店")
print("y4: 博多大博通店")

plt.figure(figsize=(10, 12))
plt.subplots_adjust(wspace=0.15,hspace=0.4)

plt.subplot(4,1,1)
ticks  = np.arange(periods)
lower  = np.zeros(periods)
height = fevd[:, 0, 0]
plt.bar(ticks, height, bottom=lower, color="green",  label="y1")

lower = lower + height
height = fevd[:, 0, 1]
plt.bar(ticks, height, bottom=lower, color="red",    label="y2")

lower = lower + height
height = fevd[:, 0, 2]
plt.bar(ticks, height, bottom=lower, color="blue",   label="y3")

lower = lower + height
height = fevd[:, 0, 3]
plt.bar(ticks, height, bottom=lower, color="yellow", label="y4")
plt.title("y1")
plt.xlabel("ラグ")
plt.grid(True)
plt.legend()

plt.subplot(4,1,2)
lower  = np.zeros(periods)
height = fevd[:, 1, 0]
plt.bar(ticks, height, bottom=lower, color="green",  label="y1")

lower = lower + height
height = fevd[:, 1, 1]
plt.bar(ticks, height, bottom=lower, color="red",    label="y2")

lower = lower + height
height = fevd[:, 1, 2]
plt.bar(ticks, height, bottom=lower, color="blue",   label="y3")

lower = lower + height
height = fevd[:, 1, 3]
plt.bar(ticks, height, bottom=lower, color="yellow", label="y4")
plt.title("y2")
plt.xlabel("ラグ")
plt.grid(True)
plt.legend()

plt.subplot(4,1,3)
lower  = np.zeros(periods)
height = fevd[:, 2, 0]
plt.bar(ticks, height, bottom=lower, color="green",  label="y1")

lower = lower + height
height = fevd[:, 2, 1]
plt.bar(ticks, height, bottom=lower, color="red",    label="y2")

lower = lower + height
height = fevd[:, 2, 2]
plt.bar(ticks, height, bottom=lower, color="blue",   label="y3")

lower = lower + height
height = fevd[:, 2, 3]
plt.bar(ticks, height, bottom=lower, color="yellow", label="y4")
plt.title("y3")
plt.xlabel("ラグ")
plt.grid(True)
plt.legend()

plt.subplot(4,1,4)
lower  = np.zeros(periods)
height = fevd[:, 3, 0]
plt.bar(ticks, height, bottom=lower, color="green",  label="y1")

lower = lower + height
height = fevd[:, 3, 1]
plt.bar(ticks, height, bottom=lower, color="red",    label="y2")

lower = lower + height
height = fevd[:, 3, 2]
plt.bar(ticks, height, bottom=lower, color="blue",   label="y3")

lower = lower + height
height = fevd[:, 3, 3]
plt.bar(ticks, height, bottom=lower, color="yellow", label="y4")
plt.title("y4")
plt.xlabel("ラグ")
plt.grid(True)
plt.legend()